In [1]:
!pip install transformers accelerate bitsandbytes peft sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 10.6 MB/s eta 0:00:00


In [2]:
!pip install -q transformers accelerate peft sentencepiece bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 15.2 MB/s eta 0:00:00


In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen2.5-0.5B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

# EASIER VERSION: no 4-bit, no bitsandbytes required
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

In [4]:
import pandas as pd
df = pd.read_csv("training_data.csv")
df.head(2)

,id,prompt,sql,language,table,complexity
0,176,Show Top 10 Highest Orders,SELECT * FROM sales_data ORDER BY total_amount...,en,sales_data,intermediate
1,291,Sabre Staff Ki List Nikalo,SELECT * FROM employee_data;,hinglish,employee_data,basic


In [7]:
# combine prompt + sql

def format_example(row):
    return f"User: {row['prompt']}\nSQL: {row['sql']}"

df["text"] = df.apply(format_example, axis=1)
len(df), df["text"].head()

(500,
 0    User: Show Top 10 Highest Orders\nSQL: SELECT ...
 1    User: Sabre Staff Ki List Nikalo\nSQL: SELECT ...
 2    User: Total Staff Count\nSQL: SELECT COUNT(*) ...
 3    User: Display Sales From East Region\nSQL: SEL...
 4    User: Display Sales From North Region\nSQL: SE...
 Name: text, dtype: object)

In [8]:
# convert and tokenize
from datasets import Dataset

dataset = Dataset.from_pandas(df[["text"]])

dataset = dataset.train_test_split(test_size=0.1, seed=42)
train_ds = dataset["train"]
eval_ds = dataset["test"]

max_length = 256

def tokenize_fn(batch):
    out = tokenizer(
        batch["text"],
        max_length=max_length,
        truncation=True,
        padding="max_length",
    )
    out["labels"] = out["input_ids"].copy()
    return out

train_tokenized = train_ds.map(tokenize_fn, batched=True, remove_columns=["text"])
eval_tokenized  = eval_ds.map(tokenize_fn, batched=True, remove_columns=["text"])


Map:   0%|          | 0/450 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

In [10]:
save_dir = "./qwen2.5-0.5b-text2sql-lora-final"

model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)

('./qwen2.5-0.5b-text2sql-lora-final/tokenizer_config.json',
 './qwen2.5-0.5b-text2sql-lora-final/special_tokens_map.json',
 './qwen2.5-0.5b-text2sql-lora-final/chat_template.jinja',
 './qwen2.5-0.5b-text2sql-lora-final/vocab.json',
 './qwen2.5-0.5b-text2sql-lora-final/merges.txt',
 './qwen2.5-0.5b-text2sql-lora-final/added_tokens.json',
 './qwen2.5-0.5b-text2sql-lora-final/tokenizer.json')

In [11]:
import os, glob

print(os.listdir(save_dir))
print("Adapter config exists:", os.path.exists(os.path.join(save_dir, "adapter_config.json")))
print("Adapter model exists:", os.path.exists(os.path.join(save_dir, "adapter_model.bin")))

['generation_config.json', 'merges.txt', 'tokenizer.json', 'added_tokens.json', 'tokenizer_config.json', 'special_tokens_map.json', 'config.json', 'vocab.json', 'chat_template.jinja', 'model.safetensors']
Adapter config exists: False
Adapter model exists: False


In [13]:
import os

save_dir = "./qwen2.5-0.5b-text2sql-lora-final"
print(os.listdir(save_dir))


['generation_config.json', 'merges.txt', 'tokenizer.json', 'added_tokens.json', 'tokenizer_config.json', 'special_tokens_map.json', 'config.json', 'vocab.json', 'chat_template.jinja', 'model.safetensors']


In [14]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

save_dir = "./qwen2.5-0.5b-text2sql-lora-final"  # folder you saved

tokenizer = AutoTokenizer.from_pretrained(save_dir, trust_remote_code=True)

model = AutoModelForCausalLM.from_pretrained(
    save_dir,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True,
)

model.eval()


Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=896, out_features=896, bias=True)
          (k_proj): Linear(in_features=896, out_features=128, bias=True)
          (v_proj): Linear(in_features=896, out_features=128, bias=True)
          (o_proj): Linear(in_features=896, out_features=896, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
          (up_proj): Linear(in_features=896, out_features=4864, bias=False)
          (down_proj): Linear(in_features=4864, out_features=896, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((896,), eps=1e-06)
    (rotary_emb): Qwen2

In [15]:
def generate_sql(nl_prompt, max_new_tokens=64):
    input_text = f"User: {nl_prompt}\nSQL:"

    inputs = tokenizer(input_text, return_tensors="pt").to(model.device)

    with torch.no_grad():
        output_ids = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id,
        )

    full_output = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    if "SQL:" in full_output:
        sql_part = full_output.split("SQL:", 1)[1].strip()
    else:
        sql_part = full_output.strip()
    return sql_part, full_output

# Test
sql_only, full = generate_sql("Sabre staff ki list nikalo")
print("Generated SQL:\n", sql_only)
print("\nFULL GENERATION:\n", full)


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Generated SQL:
 SELECT * FROM sabre_staff WHERE department = 'IT' AND position = 'Manager'

Explanation:
The SQL query selects all columns from the table "sabre_staff" where the department is 'IT' and the position is 'Manager'. This will give us a list of employees who are in the IT department and

FULL GENERATION:
 User: Sabre staff ki list nikalo
SQL: SELECT * FROM sabre_staff WHERE department = 'IT' AND position = 'Manager'

Explanation:
The SQL query selects all columns from the table "sabre_staff" where the department is 'IT' and the position is 'Manager'. This will give us a list of employees who are in the IT department and


In [16]:
import numpy as np

subset = df.sample(n=min(20, len(df)), random_state=42)

correct = 0
total = 0

for _, row in subset.iterrows():
    nl = row["prompt"]
    gold_sql = str(row["sql"]).strip()

    pred_sql, full = generate_sql(nl, max_new_tokens=64)
    pred_sql = pred_sql.strip()

    is_match = (pred_sql == gold_sql)
    correct += int(is_match)
    total += 1

    print("NL:   ", nl)
    print("GOLD: ", gold_sql)
    print("PRED: ", pred_sql)
    print("MATCH:", is_match)
    print("-" * 70)

acc = correct / max(total, 1)
print(f"\nExact match accuracy on {total} examples: {acc:.2%}")


NL:    Orders Ka Saara Data Chahiye
GOLD:  SELECT * FROM sales_data;
PRED:  SELECT * FROM `orders` WHERE `customer_id` = '12345' AND `status` = 'completed';

SELECT * FROM `orders` WHERE `customer_id` = '12345' AND `status` = 'completed' ORDER BY `order_date` DESC LIMIT 1
MATCH: False
----------------------------------------------------------------------
NL:    Employees hired in 2020
GOLD:  SELECT * FROM employee_data WHERE hire_date LIKE '2020%';
PRED:  SELECT COUNT(*) FROM employees WHERE hire_date BETWEEN '2020-01-01' AND '2020-12-31'

This SQL query will count the number of employees who were hired between January 1, 2020 and December 31, 2
MATCH: False
----------------------------------------------------------------------
NL:    Show Orders From East Region
GOLD:  SELECT * FROM sales_data WHERE region = 'East';
PRED:  SELECT * FROM orders WHERE region = 'East'

This SQL query selects all columns from the "orders" table where the "region" column is equal to 'East'. This will retur

In [17]:
save_dir = "./qwen2.5-0.5b-text2sql-final"

model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)

('./qwen2.5-0.5b-text2sql-final/tokenizer_config.json',
 './qwen2.5-0.5b-text2sql-final/special_tokens_map.json',
 './qwen2.5-0.5b-text2sql-final/chat_template.jinja',
 './qwen2.5-0.5b-text2sql-final/vocab.json',
 './qwen2.5-0.5b-text2sql-final/merges.txt',
 './qwen2.5-0.5b-text2sql-final/added_tokens.json',
 './qwen2.5-0.5b-text2sql-final/tokenizer.json')

In [18]:
while True:
    nl = input("Enter your question (or 'quit'): ")
    if nl.lower() in ["quit", "exit"]:
        break
    sql, full = generate_sql(nl)
    print("\nGenerated SQL:")
    print(sql)
    print("-" * 70)

Enter your question (or 'quit'): give all rows

Generated SQL:
SELECT * FROM table_name WHERE column1 = 'value' AND column2 = 'value'

To modify the SQL query to include a WHERE clause with an additional condition, you can simply add another AND clause after the first one. Here's the modified query:

```sql
SELECT * 
FROM table_name 
WHERE column
----------------------------------------------------------------------
Enter your question (or 'quit'): quit
